In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
yearly_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Buyuk_Veri/Dataset/global_incidents.csv')


In [ ]:
if 'start_date' in yearly_df.columns:
    yearly_df['start_date'] = pd.to_datetime(yearly_df['start_date'], dayfirst=True, errors='coerce')
    yearly_df['year'] = yearly_df['start_date'].dt.year

In [ ]:
yearly_df_filtered = yearly_df.dropna(subset=['year'])

In [ ]:
yearly_counts = yearly_df_filtered.groupby('year').size().reset_index(name='attack_count')
print(yearly_counts)

      year  attack_count
0   2000.0             3
1   2001.0             6
2   2002.0             1
3   2003.0             2
4   2004.0             4
5   2005.0             4
6   2006.0            14
7   2007.0            30
8   2008.0            29
9   2009.0            31
10  2010.0            34
11  2011.0           140
12  2012.0           119
13  2013.0           147
14  2014.0           107
15  2015.0           131
16  2016.0           166
17  2017.0           135
18  2018.0           120
19  2019.0            91
20  2020.0            71
21  2021.0           160
22  2022.0           354
23  2023.0           723
24  2024.0           700


In [ ]:
yearly_counts.to_csv('/content/drive/MyDrive/Colab Notebooks/Buyuk_Veri/Dataset/yearly_attack_counts.csv', index=False)
print("Yıllara göre saldırı sayıları CSV olarak kaydedildi.")

Yıllara göre saldırı sayıları CSV olarak kaydedildi.


In [ ]:
sector_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Buyuk_Veri/Dataset/entities.csv')

print(sector_df.info())
print(sector_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12180 entries, 0 to 12179
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           12180 non-null  object
 1   incident_id  12180 non-null  int64 
 2   name         5797 non-null   object
 3   country      12177 non-null  object
 4   category     12177 non-null  object
 5   subcategory  12177 non-null  object
dtypes: int64(1), object(5)
memory usage: 571.1+ KB
None
    id  incident_id           name     country  \
0  2_0            2  Not available     Armenia   
1  2_1            2  Not available     Armenia   
2  2_2            2  Not available     Armenia   
3  3_0            3  Not available  Azerbaijan   
4  3_1            3  Not available  Azerbaijan   

                                   category  \
0     State institutions / political system   
1                                     Media   
2                                     Other   
3                 

In [ ]:
sector_df_clean = sector_df.dropna(subset=['category']).copy()

sector_df_clean['category'] = sector_df_clean['category'].str.lower().str.strip()

print(sector_df_clean[['category']].head())

                                   category
0     state institutions / political system
1                                     media
2                                     other
3                             social groups
4  end user(s) / specially protected groups


In [ ]:
sector_counts = sector_df_clean.groupby('category')['incident_id'].nunique().reset_index(name='attack_count')

print(sector_counts.sort_values(by='attack_count', ascending=False).head(10))

                                             category  attack_count
10              state institutions / political system          1730
1                             critical infrastructure          1345
0   corporate targets (corporate targets only code...           528
5                                               media           229
9                                       social groups           222
2                                           education           189
11                                            unknown           117
3            end user(s) / specially protected groups           109
8                                             science            99
4          international / supranational organization            80


In [ ]:
sector_top15 = sector_counts.sort_values(by='attack_count', ascending=False).head(15)

sector_top15.to_csv('/content/drive/MyDrive/Colab Notebooks/Buyuk_Veri/Dataset/sector_attack_counts.csv', index=False)

print("Sektör bazlı saldırı sayıları CSV olarak kaydedildi.")

Sektör bazlı saldırı sayıları CSV olarak kaydedildi.


In [ ]:
import pandas as pd
import numpy as np
file_path = '/content/drive/MyDrive/Colab Notebooks/Buyuk_Veri/Dataset/StealthPhisher.csv'


In [ ]:
try:
    df = pd.read_csv(file_path)
    print("\u2705 Veri başarıyla yüklendi.")
except Exception as e:
    print(f"\u274c Veri yüklenirken hata oluştu: {e}")

print("\nSütunlar:", df.columns.tolist())
print("\nVeri örnekleri:")
print(df.head())

missing = df.isnull().sum()
print("\nEksik veriler:\n", missing[missing > 0])

print("\nVeri tipleri:\n", df.dtypes)

if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'], inplace=True)

categoricals = [col for col in df.columns if df[col].dtype == 'object']
print("\nKategorik sütunlar:", categoricals)


df['Label'] = df['Label'].map({'Phishing': 1, 'Legitimate': 0})

num_cols = df.select_dtypes(include=[np.number]).columns
for col in num_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].mean(), inplace=True)

from sklearn.preprocessing import StandardScaler

numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

df['Label'] = df['Label'].map({'Phishing': 1, 'Legitimate': 0})

for col in numeric_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].mean(), inplace=True)

scaler = StandardScaler()
scaled = scaler.fit_transform(df[numeric_cols])

df_scaled = pd.DataFrame(scaled, columns=numeric_cols)

df_scaled['Label'] = df['Label'].values

print("✅ Ölçekleme tamamlandı. Yeni veri örnekleri:")
print(df_scaled.head())

✅ Veri başarıyla yüklendi.

Sütunlar: ['URL', 'LengthOfURL', 'Domain', 'URLComplexity', 'CharacterComplexity', 'DomainLengthOfURL', 'IsDomainIP', 'TLD', 'TLDLength', 'LetterCntInURL', 'URLLetterRatio', 'DigitCntInURL', 'URLDigitRatio', 'EqualCharCntInURL', 'QuesMarkCntInURL', 'AmpCharCntInURL', 'OtherSpclCharCntInURL', 'URLOtherSpclCharRatio', 'NumberOfHashtags', 'NumberOfSubdomains', 'HavingPath', 'PathLength', 'HavingQuery', 'HavingFragment', 'HavingAnchor', 'HasSSL', 'IsUnreachable', 'LineOfCode', 'LongestLineLength', 'HasTitle', 'HasFavicon', 'HasRobotsBlocked', 'IsResponsive', 'IsURLRedirects', 'IsSelfRedirects', 'HasDescription', 'HasPopup', 'HasIFrame', 'IsFormSubmitExternal', 'HasSocialMediaPage', 'HasSubmitButton', 'HasHiddenFields', 'HasPasswordFields', 'HasBankingKey', 'HasPaymentKey', 'HasCryptoKey', 'HasCopyrightInfoKey ', 'CntImages', 'CntFilesCSS', 'CntFilesJS', 'CntSelfHRef', 'CntEmptyRef', 'CntExternalRef', 'CntPopup', 'CntIFrame', 'UniqueFeatureCnt', 'WAPLegitimate', 

<ipython-input-15-3fffc5d8a385>:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count


✅ Ölçekleme tamamlandı. Yeni veri örnekleri:
   LengthOfURL  URLComplexity  CharacterComplexity  DomainLengthOfURL  \
0     1.627159       0.455381            -1.596187           5.376850   
1     0.020349       0.211448            -0.717222          -0.148671   
2    -0.417872      -0.154452             0.292567          -0.247341   
3    -0.308317      -0.108714            -0.134651           0.147339   
4    -0.600464      -0.535597             1.147004          -0.642021   

   IsDomainIP  TLDLength  LetterCntInURL  URLLetterRatio  DigitCntInURL  \
0   -0.232480   0.136044        1.483594        0.839098       2.116865   
1    4.301449   5.425564       -0.615435       -2.267763       2.116865   
2   -0.232480   0.136044       -0.382210       -0.073783      -0.145762   
3   -0.232480   0.136044       -0.102339        1.007865      -0.447445   
4   -0.232480  -0.921861       -0.662080       -1.186115      -0.447445   

   URLDigitRatio  ...  UniqueFeatureCnt  WAPLegitimate  WAPPhishi

/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Buyuk_Veri/Dataset/StealthPhisher.csv'
df = pd.read_csv(file_path)
print(df['Label'].unique())

['Phishing' 'Legitimate']


In [ ]:
print(df.columns.tolist())

['URL', 'LengthOfURL', 'Domain', 'URLComplexity', 'CharacterComplexity', 'DomainLengthOfURL', 'IsDomainIP', 'TLD', 'TLDLength', 'LetterCntInURL', 'URLLetterRatio', 'DigitCntInURL', 'URLDigitRatio', 'EqualCharCntInURL', 'QuesMarkCntInURL', 'AmpCharCntInURL', 'OtherSpclCharCntInURL', 'URLOtherSpclCharRatio', 'NumberOfHashtags', 'NumberOfSubdomains', 'HavingPath', 'PathLength', 'HavingQuery', 'HavingFragment', 'HavingAnchor', 'HasSSL', 'IsUnreachable', 'LineOfCode', 'LongestLineLength', 'HasTitle', 'HasFavicon', 'HasRobotsBlocked', 'IsResponsive', 'IsURLRedirects', 'IsSelfRedirects', 'HasDescription', 'HasPopup', 'HasIFrame', 'IsFormSubmitExternal', 'HasSocialMediaPage', 'HasSubmitButton', 'HasHiddenFields', 'HasPasswordFields', 'HasBankingKey', 'HasPaymentKey', 'HasCryptoKey', 'HasCopyrightInfoKey ', 'CntImages', 'CntFilesCSS', 'CntFilesJS', 'CntSelfHRef', 'CntEmptyRef', 'CntExternalRef', 'CntPopup', 'CntIFrame', 'UniqueFeatureCnt', 'WAPLegitimate', 'WAPPhishing', 'ShannonEntropy', 'Frac

In [ ]:
df['Label'] = df['Label'].map({'Phishing': 1, 'Legitimate': 0})
print(df['Label'].value_counts())

Label
1    175806
0    160943
Name: count, dtype: int64


In [ ]:
df['TLD'] = df['TLD'].fillna('unknown')

In [ ]:
from sklearn.preprocessing import StandardScaler

categorical = ['URL', 'Domain', 'TLD']
numerical = [col for col in df.columns if col not in categorical + ['Label']]

features = df[numerical]
scaler = StandardScaler()
scaled = scaler.fit_transform(features)

In [ ]:
import pandas as pd

df_scaled = pd.DataFrame(scaled, columns=numerical)
df_scaled['Label'] = df['Label'].values

In [ ]:
output_path = '/content/drive/MyDrive/Colab Notebooks/Buyuk_Veri/Dataset/cleaned_dataset1.csv'

df_scaled.to_csv(output_path, index=False)

print(f"✅ Temizlenmiş ve ölçeklenmiş veri seti kaydedildi: {output_path}")


✅ Temizlenmiş ve ölçeklenmiş veri seti kaydedildi: /content/drive/MyDrive/Colab Notebooks/Buyuk_Veri/Dataset/cleaned_dataset1.csv


In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/Colab Notebooks/Buyuk_Veri/Dataset/cleaned_dataset1.csv'

df = pd.read_csv(file_path)

print(f"🔢 Toplam kayıt sayısı: {df.shape[0]}")

if 'Label' in df.columns:
    print("\n🎯 Label dağılımı:")
    print(df['Label'].value_counts())
else:
    print("\n⚠️ 'Label' sütunu bulunamadı.")

print("\n🧩 Sütunlar:")
print(df.columns.tolist())

print("\n🔍 Örnek veriler:")
print(df.head())


🔢 Toplam kayıt sayısı: 336749

🎯 Label dağılımı:
Label
1    175806
0    160943
Name: count, dtype: int64

🧩 Sütunlar:
['LengthOfURL', 'URLComplexity', 'CharacterComplexity', 'DomainLengthOfURL', 'IsDomainIP', 'TLDLength', 'LetterCntInURL', 'URLLetterRatio', 'DigitCntInURL', 'URLDigitRatio', 'EqualCharCntInURL', 'QuesMarkCntInURL', 'AmpCharCntInURL', 'OtherSpclCharCntInURL', 'URLOtherSpclCharRatio', 'NumberOfHashtags', 'NumberOfSubdomains', 'HavingPath', 'PathLength', 'HavingQuery', 'HavingFragment', 'HavingAnchor', 'HasSSL', 'IsUnreachable', 'LineOfCode', 'LongestLineLength', 'HasTitle', 'HasFavicon', 'HasRobotsBlocked', 'IsResponsive', 'IsURLRedirects', 'IsSelfRedirects', 'HasDescription', 'HasPopup', 'HasIFrame', 'IsFormSubmitExternal', 'HasSocialMediaPage', 'HasSubmitButton', 'HasHiddenFields', 'HasPasswordFields', 'HasBankingKey', 'HasPaymentKey', 'HasCryptoKey', 'HasCopyrightInfoKey ', 'CntImages', 'CntFilesCSS', 'CntFilesJS', 'CntSelfHRef', 'CntEmptyRef', 'CntExternalRef', 'CntPo

In [ ]:
import pandas as pd

email_path = "/content/drive/MyDrive/Colab Notebooks/Buyuk_Veri/Dataset/email_dataset.csv"


df_email = pd.read_csv(email_path, sep=';', encoding='utf-8', on_bad_lines='skip')


print("🔢 Satır sayısı:", len(df_email))
print("\n🧩 Sütunlar:", df_email.columns.tolist())
print("\n🎯 Etiket Dağılımı:\n", df_email['label'].value_counts())
print("\n🔍 Örnek Veri:\n", df_email.sample(3))

🔢 Satır sayısı: 3633

🧩 Sütunlar: ['src', 'content', 'label']

🎯 Etiket Dağılımı:
 label
benign        2370
sextortion     732
ransomware     280
blackmail      251
Name: count, dtype: int64

🔍 Örnek Veri:
          src                                            content       label
296   reddit   This is your badluck. I know iamniot is your ...  sextortion
1038  reddit  Subject : pervert - [pwd_redacted] Hi, I know ...  ransomware
2     reddit  Well i'm going to ensure it is as fast and sho...   blackmail


In [ ]:
df_email['label'] = df_email['label'].str.lower().str.strip()

df_email['label'] = df_email['label'].apply(lambda x: 0 if x == 'benign' else 1)

In [ ]:
df_email['content'] = df_email['content'].str.lower().str.strip()

In [ ]:
df_email = df_email.dropna(subset=['content', 'label'])

In [ ]:
print("✅ Temizlenmiş veri:")
print(df_email.info())
print(df_email['label'].value_counts())
print(df_email.sample(3))

✅ Temizlenmiş veri:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3633 entries, 0 to 3632
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   src      3633 non-null   object
 1   content  3633 non-null   object
 2   label    3633 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 85.3+ KB
None
label
0    2370
1    1263
Name: count, dtype: int64
         src                                            content  label
2400   Enron                                        job posting      0
406   reddit  gabriel aguilar <rdaigwandasz@hotmail.com> let...      1
1156   locky  from: herbalife <mds_adreddy_101@microsoft.com...      1


In [ ]:
df_email.to_csv('/content/drive/MyDrive/Colab Notebooks/Buyuk_Veri/Dataset/email_dataset_clean.csv', index=False)
print("💾 Temizlenmiş e-mail verisi kaydedildi.")

💾 Temizlenmiş e-mail verisi kaydedildi.


In [ ]:
import pandas as pd

sms_path = '/content/drive/MyDrive/Colab Notebooks/Buyuk_Veri/Dataset/sms_dataset.csv'

df_sms = pd.read_csv(sms_path, encoding='utf-8', sep=',', on_bad_lines='skip')

print("📏 Satır sayısı:", len(df_sms))
print("\n🧩 Sütunlar:", df_sms.columns.tolist())
print("\n🔍 İlk 5 Satır:\n", df_sms.head())


📏 Satır sayısı: 5971

🧩 Sütunlar: ['LABEL', 'TEXT', 'URL', 'EMAIL', 'PHONE']

🔍 İlk 5 Satır:
       LABEL                                               TEXT URL EMAIL PHONE
0       ham  Your opinion about me? 1. Over 2. Jada 3. Kusr...  No    No    No
1       ham  What's up? Do you want me to come online? If y...  No    No    No
2       ham                       So u workin overtime nigpun?  No    No    No
3       ham  Also sir, i sent you an email about how to log...  No    No    No
4  Smishing  Please Stay At Home. To encourage the notion o...  No    No    No


In [ ]:
df_sms['LABEL'] = df_sms['LABEL'].astype(str).str.lower().str.strip()

In [ ]:
df_sms['label'] = df_sms['LABEL'].apply(lambda x: 1 if x in ['spam', 'smishing'] else 0)

In [ ]:
df_sms['text'] = df_sms['TEXT'].astype(str).str.lower().str.strip()

In [ ]:
df_sms = df_sms[df_sms['text'].str.strip() != '']
df_sms = df_sms.dropna(subset=['text', 'label'])

In [ ]:
print("📌 Kalan veri satırı sayısı:", len(df_sms))
print("🎯 Etiket dağılımı:\n", df_sms['label'].value_counts())

📌 Kalan veri satırı sayısı: 5971
🎯 Etiket dağılımı:
 label
0    4844
1    1127
Name: count, dtype: int64


In [ ]:
df_sms[['text', 'label']].to_csv('/content/drive/MyDrive/Colab Notebooks/Buyuk_Veri/Dataset/sms_dataset_clean.csv', index=False)
print("✅ Veri kaybı olmadan temizlenmiş SMS verisi kaydedildi.")

✅ Veri kaybı olmadan temizlenmiş SMS verisi kaydedildi.


In [ ]:
import pandas as pd

url_path = "/content/drive/MyDrive/Colab Notebooks/Buyuk_Veri/Dataset/url_dataset.csv"

try:
    df_url = pd.read_csv(url_path, sep=',', encoding='utf-8', on_bad_lines='skip', quotechar='"')
    print("✅ `url_dataset.csv` virgül (,) ayırıcı ve tırnak işareti ile başarıyla okundu.")
except Exception as e:
    print(f"❌ Virgül (,) ayırıcı ile okuma hatası: {e}. Farklı ayırıcıları veya parametreleri deneyin.")
    # Örneğin: !head -n 5 "/content/drive/MyDrive/Colab Notebooks/Buyuk_Veri/Dataset/cleaned_url.csv"
    exit()

print("\n🔢 Satır sayısı:", len(df_url))
print("\n🧩 Sütunlar (DÜZELTİLMİŞ HALİ):")
print(df_url.columns.tolist())
print("\n🔍 Örnek Veri (DÜZELTİLMİŞ HALİ):\n", df_url.sample(3))

✅ `url_dataset.csv` virgül (,) ayırıcı ve tırnak işareti ile başarıyla okundu.

🔢 Satır sayısı: 61239

🧩 Sütunlar (DÜZELTİLMİŞ HALİ):
['LengthOfURL', 'URLComplexity', 'CharacterComplexity', 'DomainLengthOfURL', 'IsDomainIP', 'TLDLength', 'LetterCntInURL', 'URLLetterRatio', 'DigitCntInURL', 'URLDigitRatio', 'EqualCharCntInURL', 'QuesMarkCntInURL', 'AmpCharCntInURL', 'OtherSpclCharCntInURL', 'URLOtherSpclCharRatio', 'NumberOfHashtags', 'NumberOfSubdomains', 'HavingPath', 'PathLength', 'HavingQuery', 'HavingFragment', 'HavingAnchor', 'HasSSL', 'IsUnreachable', 'LineOfCode', 'LongestLineLength', 'HasTitle', 'HasFavicon', 'HasRobotsBlocked', 'IsResponsive', 'IsURLRedirects', 'IsSelfRedirects', 'HasDescription', 'HasPopup', 'HasIFrame', 'IsFormSubmitExternal', 'HasSocialMediaPage', 'HasSubmitButton', 'HasHiddenFields', 'HasPasswordFields', 'HasBankingKey', 'HasPaymentKey', 'HasCryptoKey', 'HasCopyrightInfoKey ', 'CntImages', 'CntFilesCSS', 'CntFilesJS', 'CntSelfHRef', 'CntEmptyRef', 'CntExte

In [ ]:

if 'Label' in df_url.columns:
    df_url['Label'] = pd.to_numeric(df_url['Label'], errors='coerce')


    if df_url['Label'].isnull().any():
        most_frequent_label = df_url['Label'].mode()[0]
        df_url['Label'].fillna(most_frequent_label, inplace=True)
        print(f"✅ 'Label' sütunundaki NaN değerler en sık görülen değer ({int(most_frequent_label)}) ile dolduruldu.")

    df_url['Label'] = df_url['Label'].astype(int)
    print("✅ 'Label' sütunu temizlendi ve sayısal (int) hale getirildi.")
    print("Yeni 'Label' dağılımı:\n", df_url['Label'].value_counts())
else:
    print("⚠️ 'Label' sütunu bulunamadı. Saldırı tespiti için bu sütun gereklidir.")


if 'TLD' in df_url.columns and df_url['TLD'].dtype == 'object' and df_url['TLD'].isnull().any():
    df_url['TLD'].fillna('unknown', inplace=True)
    print("✅ 'TLD' sütunundaki eksik değerler 'unknown' ile dolduruldu.")
elif 'TLD' in df_url.columns and df_url['TLD'].isnull().any():
     most_frequent_tld = df_url['TLD'].mode()[0]
     df_url['TLD'].fillna(most_frequent_tld, inplace=True)
     print(f"✅ Sayısal 'TLD' sütunundaki eksik değerler en sık görülen değer ({int(most_frequent_tld)}) ile dolduruldu.")


print("\nTemizleme sonrası URL veri örneği:")
print(df_url.head())

✅ 'Label' sütunu temizlendi ve sayısal (int) hale getirildi.
Yeni 'Label' dağılımı:
 Label
1    32151
0    29088
Name: count, dtype: int64

Temizleme sonrası URL veri örneği:
   LengthOfURL  URLComplexity  CharacterComplexity  DomainLengthOfURL  \
0     1.627159       0.455381            -1.596187           5.376850   
1     0.020349       0.211448            -0.717222          -0.148671   
2    -0.417872      -0.154452             0.292567          -0.247341   
3    -0.308317      -0.108714            -0.134651           0.147339   
4    -0.600464      -0.535597             1.147004          -0.642021   

   IsDomainIP  TLDLength  LetterCntInURL  URLLetterRatio  DigitCntInURL  \
0   -0.232480   0.136044        1.483594        0.839098       2.116865   
1    4.301449   5.425564       -0.615435       -2.267763       2.116865   
2   -0.232480   0.136044       -0.382210       -0.073783      -0.145762   
3   -0.232480   0.136044       -0.102339        1.007865      -0.447445   
4   -0.2324

In [ ]:
initial_rows = len(df_url)
df_url.drop_duplicates(inplace=True)
rows_after_dropping = len(df_url)

if initial_rows > rows_after_dropping:
    print(f"✅ Yinelenen satırlar bulundu ve kaldırıldı. Başlangıç: {initial_rows} satır, Son: {rows_after_dropping} satır.")
    print(f"Toplam {initial_rows - rows_after_dropping} adet yinelenen satır kaldırıldı.")
else:
    print("ℹ️ URL veri setinde yinelenen satır bulunamadı.")

print("\nYinelenen satırlar kaldırıldıktan sonraki URL veri örneği:")
print(df_url.head())

✅ Yinelenen satırlar bulundu ve kaldırıldı. Başlangıç: 61239 satır, Son: 61048 satır.
Toplam 191 adet yinelenen satır kaldırıldı.

Yinelenen satırlar kaldırıldıktan sonraki URL veri örneği:
   LengthOfURL  URLComplexity  CharacterComplexity  DomainLengthOfURL  \
0     1.627159       0.455381            -1.596187           5.376850   
1     0.020349       0.211448            -0.717222          -0.148671   
2    -0.417872      -0.154452             0.292567          -0.247341   
3    -0.308317      -0.108714            -0.134651           0.147339   
4    -0.600464      -0.535597             1.147004          -0.642021   

   IsDomainIP  TLDLength  LetterCntInURL  URLLetterRatio  DigitCntInURL  \
0   -0.232480   0.136044        1.483594        0.839098       2.116865   
1    4.301449   5.425564       -0.615435       -2.267763       2.116865   
2   -0.232480   0.136044       -0.382210       -0.073783      -0.145762   
3   -0.232480   0.136044       -0.102339        1.007865      -0.447445

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np

numeric_cols_url = df_url.select_dtypes(include=[np.number]).columns.tolist()

if 'Label' in numeric_cols_url:
    numeric_cols_url.remove('Label')

if len(numeric_cols_url) > 0:
    scaler_url = StandardScaler()
    df_url[numeric_cols_url] = scaler_url.fit_transform(df_url[numeric_cols_url])
    print("✅ URL veri setindeki sayısal özellikler ölçeklendi.")
else:
    print("⚠️ URL veri setinde ölçeklenecek sayısal özellik bulunamadı (Label dışındaki).")

print("\nÖlçekleme sonrası URL veri örneği:")
print(df_url.head())

✅ URL veri setindeki sayısal özellikler ölçeklendi.

Ölçekleme sonrası URL veri örneği:
   LengthOfURL  URLComplexity  CharacterComplexity  DomainLengthOfURL  \
0     1.593718       0.441088            -1.588934           5.362426   
1     0.011480       0.200827            -0.709800          -0.154308   
2    -0.420040      -0.159565             0.300183          -0.252822   
3    -0.312160      -0.114516            -0.127118           0.141231   
4    -0.599839      -0.534973             1.154783          -0.646874   

   IsDomainIP  TLDLength  LetterCntInURL  URLLetterRatio  DigitCntInURL  \
0   -0.235507   0.127254        1.453747        0.832136       2.078853   
1    4.246166   5.388393       -0.613514       -2.251303       2.078853   
2   -0.235507   0.127254       -0.383819       -0.073863      -0.151828   
3   -0.235507   0.127254       -0.108184        0.999631      -0.449252   
4   -0.235507  -0.924974       -0.659453       -1.177810      -0.449252   

   URLDigitRatio  ... 

In [ ]:

output_path_cleaned_url_final = '/content/drive/MyDrive/Colab Notebooks/Buyuk_Veri/Dataset/url_dataset_clean.csv'
df_url.to_csv(output_path_cleaned_url_final, index=False)
print(f"\n✅ Temizlenmiş ve ön işlenmiş URL verisi başarıyla kaydedildi: {output_path_cleaned_url_final}")


✅ Temizlenmiş ve ön işlenmiş URL verisi başarıyla kaydedildi: /content/drive/MyDrive/Colab Notebooks/Buyuk_Veri/Dataset/url_dataset_clean.csv


Bu adıma kadar hazır veri setleri üzerinde veri ön işleme işlemleri gerçekleştirilmiştir. Bundan sonraki adımda, analizler için ihtiyaç duyacağımız yaş ve cinsiyet gibi bilgiler içeren bir veri setini suni olarak oluşturacağız.

In [ ]:
!pip install Faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.0 MB/s eta 0:00:00


In [5]:
!pip install Faker

import pandas as pd
import numpy as np
from faker import Faker
import random

fake = Faker('en_US')
num_records = 100_000

# Sabit kategoriler
yas_gruplari = ['18-25', '26-35', '36-45', '46-55', '55+']
cinsiyetler = ['Erkek', 'Kadın', 'Diğer']
egitim_durumlari = ['Lise', 'Lisans', 'Yuksek Lisans', 'Doktora']
meslek_sektorleri = ['Finans', 'Teknoloji', 'Saglik', 'Egitim', 'Kamu', 'Perakende', 'Uretim', 'Telekomunikasyon', 'Enerji', 'Diger']
saldiri_turleri = ['Kimlik Avı Epostası', 'SMS Phishing', 'Vishing', 'Pretexting', 'Oltalama (Whaling)', 'Kuyruk Avcılığı (Tailgating)', 'Yok']
farkindalik_duzeyleri = [1, 2, 3, 4, 5]

data = []

for _ in range(num_records):
    yas_grubu = random.choices(yas_gruplari, weights=[0.25, 0.30, 0.25, 0.15, 0.05])[0]
    cinsiyet = random.choices(cinsiyetler, weights=[0.48, 0.48, 0.04])[0]
    egitim_durumu = random.choices(egitim_durumlari, weights=[0.2, 0.4, 0.25, 0.15])[0]
    meslek_sektoru = random.choices(meslek_sektorleri, weights=[0.15, 0.15, 0.10, 0.10, 0.10, 0.10, 0.10, 0.05, 0.05, 0.10])[0]
    farkindalik_duzeyi = random.choices(farkindalik_duzeyleri, weights=[0.1, 0.15, 0.25, 0.3, 0.2])[0]

    # Farkındalık → davranış
    if farkindalik_duzeyi >= 4:
        kullanici_davranisi = random.choices(['Dikkatli', 'Ortalama'], weights=[0.7, 0.3])[0]
    elif farkindalik_duzeyi == 3:
        kullanici_davranisi = 'Ortalama'
    else:
        kullanici_davranisi = random.choices(['Ortalama', 'Dikkatsiz'], weights=[0.4, 0.6])[0]

    # ⚙ Yeni: Maruz kalma olasılığı ↑
    maruz_kalma_olasiligi = 0.45
    if yas_grubu in ['18-25', '55+']:
        maruz_kalma_olasiligi += 0.07
    if meslek_sektoru in ['Finans', 'Teknoloji', 'Kamu']:
        maruz_kalma_olasiligi += 0.10
    if kullanici_davranisi == 'Dikkatsiz':
        maruz_kalma_olasiligi += 0.15
    if farkindalik_duzeyi <= 2:
        maruz_kalma_olasiligi += 0.12

    maruz_kalma_olasiligi = min(max(maruz_kalma_olasiligi, 0.1), 0.95)
    saldiriya_maruz_kalma = 1 if random.random() < maruz_kalma_olasiligi else 0
    saldiri_turu = random.choice([t for t in saldiri_turleri if t != 'Yok']) if saldiriya_maruz_kalma else 'Yok'

    # ⚙ Yeni: Başarı olasılığı ↑
    saldiri_basari_olasiligi = 0.5
    if saldiriya_maruz_kalma:
        if farkindalik_duzeyi >= 4:
            saldiri_basari_olasiligi -= 0.20
        elif farkindalik_duzeyi <= 2:
            saldiri_basari_olasiligi += 0.25
        if kullanici_davranisi == 'Dikkatli':
            saldiri_basari_olasiligi -= 0.15
        elif kullanici_davranisi == 'Dikkatsiz':
            saldiri_basari_olasiligi += 0.20

    saldiri_basari_olasiligi = min(max(saldiri_basari_olasiligi, 0.05), 0.95)
    saldiri_sonucu = 1 if (saldiriya_maruz_kalma and random.random() < saldiri_basari_olasiligi) else 0

    data.append([
        fake.uuid4(), yas_grubu, cinsiyet, egitim_durumu, meslek_sektoru,
        saldiri_turu, saldiriya_maruz_kalma, saldiri_sonucu,
        farkindalik_duzeyi, kullanici_davranisi
    ])

# 📦 DataFrame oluştur ve kaydet
df = pd.DataFrame(data, columns=[
    'Kullanici_ID', 'Yas_Grubu', 'Cinsiyet', 'Egitim_Durumu', 'Meslek_Sektoru',
    'Saldırı_Türü', 'Saldırıya_Maruz_Kalma', 'Saldırı_Sonucu',
    'Farkindalik_Duzeyi', 'Kullanici_Davranisi'
])

df.to_csv('/content/drive/MyDrive/Colab Notebooks/Buyuk_Veri/Dataset/sentetik_demografik_saldiri_verisi.csv', index=False)
print("✅ Yeni dengeli sentetik veri seti başarıyla oluşturuldu.")


✅ Yeni dengeli sentetik veri seti başarıyla oluşturuldu.


In [3]:
df['Saldırıya_Maruz_Kalma'].value_counts()
df['Saldırı_Sonucu'].value_counts()

,count
Saldırı_Sonucu,
0,72855
1,27145


In [6]:
import pandas as pd

# Dosyayı yükle
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Buyuk_Veri/Dataset/sentetik_demografik_saldiri_verisi.csv')

# Rastgele örnekleri göster
ornekler = df.sample(10, random_state=42)
print(ornekler)

                               Kullanici_ID Yas_Grubu Cinsiyet  Egitim_Durumu  \
75721  dc441d19-d931-4160-a349-5b446baa26a8     26-35    Kadın           Lise   
80184  81f7bc9a-1369-4fe0-9ff7-449fd5b68407     26-35    Kadın         Lisans   
19864  2911b570-a3d4-45ec-a0c6-de0fb8d3475d     36-45    Diğer         Lisans   
76699  b47e7e93-d8a0-473f-b9a3-51145c412639     26-35    Kadın  Yuksek Lisans   
92991  72020262-dcc5-4e2a-a687-48de1e5f53f9     36-45    Kadın  Yuksek Lisans   
76434  78a8ae99-47a6-493b-ad8c-16e3b47200cb     36-45    Erkek         Lisans   
84004  cc130a7a-25f3-4326-b95f-96655dcb6153     46-55    Erkek        Doktora   
80917  cca4bbd7-8b78-460a-805f-3768c8d0606b     46-55    Erkek  Yuksek Lisans   
60767  57eec43f-9deb-4ce8-ac81-a04bfa3a125c     36-45    Kadın         Lisans   
50074  a321228f-a92e-4e13-9434-0e20a69c2e3e     36-45    Kadın  Yuksek Lisans   

      Meslek_Sektoru         Saldırı_Türü  Saldırıya_Maruz_Kalma  \
75721         Saglik                  Yo